# Creating the mean collision severity array 👍  

## "clean.csv" database preprocessing

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


df = pd.read_csv('raw_data/clean.csv') # depends on where you saved the csv file
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718663 entries, 0 to 718662
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   case_id             718663 non-null  int64  
 1   primary_road        718663 non-null  object 
 2   weather_1           718663 non-null  object 
 3   collision_severity  718663 non-null  int64  
 4   latitude            718663 non-null  float64
 5   longitude           718663 non-null  float64
 6   hour                718663 non-null  int64  
 7   week_of_the_year    718663 non-null  int64  
 8   day_of_the_week     718663 non-null  int64  
 9   routes              718663 non-null  object 
dtypes: float64(2), int64(5), object(3)
memory usage: 54.8+ MB


In [3]:
df.head()

,case_id,primary_road,weather_1,collision_severity,latitude,longitude,hour,week_of_the_year,day_of_the_week,routes
0,3516974,RT 101,clear,1,34.17357,-118.54336,10,2,4,Ventura Freeway
1,3522174,RT 405,clear,1,34.16803,-118.46901,6,3,3,San Diego Freeway
2,3524803,RT 101,cloudy,3,34.16938,-118.49952,6,1,3,Ventura Freeway
3,3524807,RT 101,cloudy,2,34.16968,-118.49999,6,1,3,Ventura Freeway
4,3524811,RT 118,clear,1,34.27350,-118.49221,14,1,3,Ronald Reagan Freeway


In [4]:
df = df.rename(columns = {'routes':'OSRM_API_ROAD_name','day_of_the_week':'weekday','primary_road':'dataset_ROAD_name'})
df

,case_id,dataset_ROAD_name,weather_1,collision_severity,latitude,longitude,hour,week_of_the_year,weekday,OSRM_API_ROAD_name
0,3516974,RT 101,clear,1,34.17357,-118.54336,10,2,4,Ventura Freeway
1,3522174,RT 405,clear,1,34.16803,-118.46901,6,3,3,San Diego Freeway
2,3524803,RT 101,cloudy,3,34.16938,-118.49952,6,1,3,Ventura Freeway
3,3524807,RT 101,cloudy,2,34.16968,-118.49999,6,1,3,Ventura Freeway
4,3524811,RT 118,clear,1,34.27350,-118.49221,14,1,3,Ronald Reagan Freeway
...,...,...,...,...,...,...,...,...,...,...
718658,8078566,RT 210,clear,2,34.13608,-117.36847,1,24,3,Foothill Freeway
718659,8165686,FLORENCE AV,clear,2,33.96229,-118.37351,16,41,0,Hindry Avenue
718660,90326285,CHAPMAN AVENUE (1500 BLOCK),clear,5,33.78883,-117.93185,1,49,6,Chapman Avenue
718661,7205180,RT 60,clear,1,34.03334,-118.18097,4,53,0,Pomona Freeway


In [5]:
# changing columns order to compare former and new road names
cols = df.columns.tolist()
cols = cols[:1] + cols[-1:] + cols[1:-1]
df = df[cols]
df


,case_id,OSRM_API_ROAD_name,dataset_ROAD_name,weather_1,collision_severity,latitude,longitude,hour,week_of_the_year,weekday
0,3516974,Ventura Freeway,RT 101,clear,1,34.17357,-118.54336,10,2,4
1,3522174,San Diego Freeway,RT 405,clear,1,34.16803,-118.46901,6,3,3
2,3524803,Ventura Freeway,RT 101,cloudy,3,34.16938,-118.49952,6,1,3
3,3524807,Ventura Freeway,RT 101,cloudy,2,34.16968,-118.49999,6,1,3
4,3524811,Ronald Reagan Freeway,RT 118,clear,1,34.27350,-118.49221,14,1,3
...,...,...,...,...,...,...,...,...,...,...
718658,8078566,Foothill Freeway,RT 210,clear,2,34.13608,-117.36847,1,24,3
718659,8165686,Hindry Avenue,FLORENCE AV,clear,2,33.96229,-118.37351,16,41,0
718660,90326285,Chapman Avenue,CHAPMAN AVENUE (1500 BLOCK),clear,5,33.78883,-117.93185,1,49,6
718661,7205180,Pomona Freeway,RT 60,clear,1,34.03334,-118.18097,4,53,0


In [6]:
# for now we won't be needing the week of the year, as this would shrink too much our data when querying, nor the case_id
df = df.drop(['week_of_the_year'],axis=1)
df.head()

,case_id,OSRM_API_ROAD_name,dataset_ROAD_name,weather_1,collision_severity,latitude,longitude,hour,weekday
0,3516974,Ventura Freeway,RT 101,clear,1,34.17357,-118.54336,10,4
1,3522174,San Diego Freeway,RT 405,clear,1,34.16803,-118.46901,6,3
2,3524803,Ventura Freeway,RT 101,cloudy,3,34.16938,-118.49952,6,3
3,3524807,Ventura Freeway,RT 101,cloudy,2,34.16968,-118.49999,6,3
4,3524811,Ronald Reagan Freeway,RT 118,clear,1,34.27350,-118.49221,14,3


In [7]:
df.collision_severity.value_counts()

1    472723
2    161652
3     67565
4     13001
5      3720
0         2
Name: collision_severity, dtype: int64

In [8]:
df.weather_1.value_counts()

clear      616122
cloudy      84240
raining     15968
unknown       941
fog           908
other         170
snowing       158
wind          156
Name: weather_1, dtype: int64

In [9]:
# we will (slightly arbitrarily) encode the weather type
weather = {'clear':0,'cloudy':1,'raining':2,'fog':3,'wind':2,'snowing':3,'other':0,'unknown':0}
def impute_weather(x):
    return weather[x]
df.weather_1=df.weather_1.apply(lambda x : impute_weather(x))
df.head()

,case_id,OSRM_API_ROAD_name,dataset_ROAD_name,weather_1,collision_severity,latitude,longitude,hour,weekday
0,3516974,Ventura Freeway,RT 101,0,1,34.17357,-118.54336,10,4
1,3522174,San Diego Freeway,RT 405,0,1,34.16803,-118.46901,6,3
2,3524803,Ventura Freeway,RT 101,1,3,34.16938,-118.49952,6,3
3,3524807,Ventura Freeway,RT 101,1,2,34.16968,-118.49999,6,3
4,3524811,Ronald Reagan Freeway,RT 118,0,1,34.27350,-118.49221,14,3


In [10]:
print('Out of',df.shape[0],'observations in L.A., there are',len(df.dataset_ROAD_name.value_counts()),'unique roads.')

Out of 718663 observations in L.A., there are 47949 unique roads.


####  preprocessing the road names

In [11]:
def name_proc(x):
    x=str(x).upper()
    nam = {' blvd ':' boulevard ',' bl ':' boulevard ',' rd ':' road ',' str ':' street ',' av ':' avenue ',' rt ':' route ',' blvd':' boulevard ',' bl':' boulevard ',' rd':' road ',' str':' street ',' av':' avenue',' rt':' route '}
    for k in nam.keys():
        if k in x:
            return x.replace(k,nam[k])
    else:
        return x

df.OSRM_API_ROAD_name = df.OSRM_API_ROAD_name.apply(lambda x : name_proc(x)) 


## Features (dimensions of the array) : weekday, dayhour, weather, road

#### starting with an example 🌝 

In [12]:
# say the user of our front-end streamlite website wants to leave from 'Beverly boulevard' at 4 P.M. on a thursday
# Let's find this road, see if we have it in store for that day and hour, then assess the average collision severity there
def beverly(x):
    if 'beverly' in x.lower():
        return x
    else:
        return ''
a = df.copy()
a.OSRM_API_ROAD_name = a.OSRM_API_ROAD_name.apply(lambda x:beverly(x))
a=a[(a.OSRM_API_ROAD_name != '')&(a.hour==16)&(a.weekday==4)]
a


,case_id,OSRM_API_ROAD_name,dataset_ROAD_name,weather_1,collision_severity,latitude,longitude,hour,weekday
25723,4270056,WEST BEVERLY BOULEVARD,BEVERLY BL,0,2,34.02124,-118.12599,16,4
135903,5490979,BEVERLY STREET,RT 91,0,1,33.87600,-118.11700,16,4
165486,5845229,WEST BEVERLY BOULEVARD,ATLANTIC BL,0,3,34.03224,-118.15381,16,4
333175,8397073,WEST BEVERLY BOULEVARD,WEST BEVERLY BL,0,2,34.01705,-118.10499,16,4
343373,8544754,WEST BEVERLY BOULEVARD,WEST BEVERLY BL,0,2,34.02406,-118.13924,16,4
345327,8569949,WEST BEVERLY BOULEVARD,N 10TH ST,0,1,34.01776,-118.10862,16,4
360530,8771538,BEVERLY BOULEVARD,BEVERLY BL,0,2,34.07720,-118.38889,16,4
364155,8820150,WEST BEVERLY BOULEVARD,N POPLAR AV,0,1,34.01620,-118.10091,16,4
374040,8959232,WEST BEVERLY BOULEVARD,N SPRUCE ST,0,2,34.01753,-118.10755,16,4
530235,9062274,WEST BEVERLY BOULEVARD,HOWARD AV,0,2,34.01808,-118.11016,16,4


In [13]:
# let's now refine the above search:
a.groupby('OSRM_API_ROAD_name').agg({'collision_severity':'mean'})

,collision_severity
OSRM_API_ROAD_name,
BEVERLY BOULEVARD,2.000000
BEVERLY STREET,1.000000
WEST BEVERLY BOULEVARD,1.777778


In [14]:
print('We would then select the right road and provide the mean severity. Let us create the full array now')

We would then select the right road and provide the mean severity. Let us create the full array now


In [15]:
df[(df.weekday==4)&(df.hour==2)] # applying a multi boolean mask

,case_id,OSRM_API_ROAD_name,dataset_ROAD_name,weather_1,collision_severity,latitude,longitude,hour,weekday
549,3675319,SAN DIEGO FREEWAY,RT 405,0,1,34.23398,-118.47290,2,4
827,3687217,VENTURA FREEWAY,RT 101,0,1,34.15607,-118.42605,2,4
837,3688084,SLAUSON AVENUE,SLAUSON AV,0,1,33.96481,-118.06941,2,4
934,3690905,SAN DIEGO FREEWAY,RT 405,0,1,34.23530,-118.47286,2,4
1029,3695624,TUNA CANYON ROAD,TUNA CANYON RD,0,4,34.04666,-118.59057,2,4
...,...,...,...,...,...,...,...,...,...
715113,91322461,SAN BERNARDINO FREEWAY,I-10 E/B,0,2,34.06824,-117.38293,2,4
716801,9134517,TYLER STREET,TYLER ST,0,2,33.91209,-117.46163,2,4
717394,9141850,ORTEGA HIGHWAY,BEACH BL,0,1,33.50580,-117.59500,2,4
718104,9150603,MARGARITA ROAD,MARGARITA RD,0,2,33.53093,-117.15112,2,4


### (TBD for the enhanced version of the array: find most relevant features then add them as dimensions)

## Imputing missing values

### simple imputer: replace missing values with average collision severity on the same road, regardless of features (ie even if weather, hour etc. are different)

#### say I start my journey on margarita road on a sunday at 9am:

In [16]:
sevdf =df[(df.weekday==6)&(df.hour==9)&(df.OSRM_API_ROAD_name=='margarita road')].groupby('OSRM_API_ROAD_name').agg({'collision_severity':'mean'})
sevdf

,collision_severity
OSRM_API_ROAD_name,


#### There is no data to retrieve because there was no such accident reported. Still, assuming no severity collision in the event of a crash is clearly wrong, so we need to impute a value here. We will use as a baseline an imputation based on data for the same road with different hours and days. 

In [17]:
corr_severity_hour = df.collision_severity.corr(df.hour)
corr_severity_hour

-0.010542164408588107

In [18]:
corr_severity_day = df.collision_severity.corr(df.weekday)
corr_severity_day

0.04323533204186572

In [19]:
# there is no linear correlation as seen above, but still there is some correlation

In [20]:
# let us see if by any chance we have a record at the exact time or weekday on the same road

In [21]:
df[(df.hour==9)&(df.OSRM_API_ROAD_name=='margarita road')].groupby('OSRM_API_ROAD_name').agg({'collision_severity':'mean'})

,collision_severity
OSRM_API_ROAD_name,


In [22]:
df[(df.weekday==6)&(df.OSRM_API_ROAD_name=='margarita road')].groupby('OSRM_API_ROAD_name').agg({'collision_severity':'mean'})


,collision_severity
OSRM_API_ROAD_name,


In [23]:
# we don't so let's impute in a straightforward manner

In [24]:
df[(df.OSRM_API_ROAD_name=='margarita road')].groupby('OSRM_API_ROAD_name').agg({'collision_severity':'mean'})


,collision_severity
OSRM_API_ROAD_name,


#### we will send 1 as proxy for collision severity here

### (TBD for the enhanced version of the array: KNN imputer)

In [25]:
# we will use a KNN imputer in a 4-dimensional space (lat, lon, weekday, dayhour) 
# in order to retrieve the average collision severity of the k nearest neighbours of the user's inputs

In [26]:
# first, how many unique roads are populated with at least one collision for any (hour,weekday) combination?

In [27]:
a = df.copy()
a = a.groupby('OSRM_API_ROAD_name').agg({'case_id':'count'}).sort_values('case_id',ascending=False)
print('There are',a[a['case_id']>=7*24].shape[0],'unique road names that have at least 168 observations in the dataset. But they likely not all contain all possible (hour,weekday) combinations.')

There are 335 unique road names that have at least 168 observations in the dataset. But they likely not all contain all possible (hour,weekday) combinations.


In [28]:
# step 1 : create rows for all roads that do not have the (hour,weekday) combination populated. Put "NaN" in collision_severity column
# step 2 : implement KNN imputer 

In [29]:
b=df.copy()
b

,case_id,OSRM_API_ROAD_name,dataset_ROAD_name,weather_1,collision_severity,latitude,longitude,hour,weekday
0,3516974,VENTURA FREEWAY,RT 101,0,1,34.17357,-118.54336,10,4
1,3522174,SAN DIEGO FREEWAY,RT 405,0,1,34.16803,-118.46901,6,3
2,3524803,VENTURA FREEWAY,RT 101,1,3,34.16938,-118.49952,6,3
3,3524807,VENTURA FREEWAY,RT 101,1,2,34.16968,-118.49999,6,3
4,3524811,RONALD REAGAN FREEWAY,RT 118,0,1,34.27350,-118.49221,14,3
...,...,...,...,...,...,...,...,...,...
718658,8078566,FOOTHILL FREEWAY,RT 210,0,2,34.13608,-117.36847,1,3
718659,8165686,HINDRY AVENUE,FLORENCE AV,0,2,33.96229,-118.37351,16,0
718660,90326285,CHAPMAN AVENUE,CHAPMAN AVENUE (1500 BLOCK),0,5,33.78883,-117.93185,1,6
718661,7205180,POMONA FREEWAY,RT 60,0,1,34.03334,-118.18097,4,0


In [30]:
b.groupby(['OSRM_API_ROAD_name','weather_1','hour','weekday']).mean()

case_id  collision_severity  \
OSRM_API_ROAD_name weather_1 hour weekday                                   
10TH AVENUE        0         5    6         4284186.0                 1.0   
                             11   3        90585664.0                 1.0   
                                  4        90460643.0                 1.0   
                             14   1         6717700.0                 2.0   
                             21   4        90178610.0                 1.0   
...                                               ...                 ...   
ZURICH DRIVE       0         13   0        90504107.0                 1.0   
                                  5        90461743.5                 1.0   
                             14   6         6601791.0                 2.0   
                             19   4         4864679.0                 3.0   
                             22   5        90844461.0                 4.0   

                                            latitude   longitude  
OSRM_API_ROAD_name weather_1 hour weekday                         
10TH AVENUE        0         5    6        34.035080 -118.328450  
                             11   3        34.035520 -118.328440  
                                  4        34.034510 -118.328510  
                             14   1        33.989220 -118.328250  
                             21   4        34.034690 -118.328400  
...                                              ...         ...  
ZURICH DRIVE       0         13   0        34.243840 -117.274810  
                                  5        34.244125 -117.272605  
                             14   6        34.244880 -117.277730  
                             19   4        34.244850 -117.277890  
                             22   5        34.245290 -117.280100  

[217777 rows x 4 columns]

In [31]:
b['moment'] = b.apply(lambda x: str(x[7]) + '-' + str(x[8]), axis=1)
b


,case_id,OSRM_API_ROAD_name,dataset_ROAD_name,weather_1,collision_severity,latitude,longitude,hour,weekday,moment
0,3516974,VENTURA FREEWAY,RT 101,0,1,34.17357,-118.54336,10,4,10-4
1,3522174,SAN DIEGO FREEWAY,RT 405,0,1,34.16803,-118.46901,6,3,6-3
2,3524803,VENTURA FREEWAY,RT 101,1,3,34.16938,-118.49952,6,3,6-3
3,3524807,VENTURA FREEWAY,RT 101,1,2,34.16968,-118.49999,6,3,6-3
4,3524811,RONALD REAGAN FREEWAY,RT 118,0,1,34.27350,-118.49221,14,3,14-3
...,...,...,...,...,...,...,...,...,...,...
718658,8078566,FOOTHILL FREEWAY,RT 210,0,2,34.13608,-117.36847,1,3,1-3
718659,8165686,HINDRY AVENUE,FLORENCE AV,0,2,33.96229,-118.37351,16,0,16-0
718660,90326285,CHAPMAN AVENUE,CHAPMAN AVENUE (1500 BLOCK),0,5,33.78883,-117.93185,1,6,1-6
718661,7205180,POMONA FREEWAY,RT 60,0,1,34.03334,-118.18097,4,0,4-0


In [32]:
# let us get a UNIQUE LIST OF ROADS (in the API)
roads = b[['OSRM_API_ROAD_name']].drop_duplicates()
roads = roads.OSRM_API_ROAD_name.to_list()
roads

['VENTURA FREEWAY',
 'SAN DIEGO FREEWAY',
 'RONALD REAGAN FREEWAY',
 'DEODAR STREET',
 'BRIGGS ROAD',
 'CHINO VALLEY FREEWAY',
 'TEMECULA VALLEY FREEWAY',
 'VANOWEN STREET',
 'SEPULVEDA BOULEVARD',
 'SANTA ANA FREEWAY',
 'SANTA MONICA FREEWAY',
 'EL PRADO AVENUE',
 'PACIFIC COAST HIGHWAY',
 'FOOTHILL FREEWAY',
 'GARDEN GROVE FREEWAY',
 'EAST VILLA VISTA WAY',
 'WILSON STREET',
 'KALISHER STREET',
 'COSTA MESA FREEWAY',
 'RIVERSIDE FREEWAY',
 'BROCKWAY STREET',
 'VENTURA BOULEVARD',
 'COLDWATER CANYON AVENUE',
 'SAN BERNARDINO FREEWAY',
 'ORANGE BLOSSOM AVENUE',
 'CAMPUS AVENUE',
 'ETIWANDA AVENUE',
 'MONROE STREET',
 'WEST KATELLA AVENUE',
 'ORANGE FREEWAY',
 'SAN BERNARDINO AVENUE',
 'LINDERNBERGER ROAD',
 'SAN GABRIEL RIVER FREEWAY',
 'SHERMAN WAY',
 'CALABASAS ROAD',
 'HIDDEN SPRINGS ROAD',
 'FOOTHILL TRANSPORTATION CORRIDOR',
 'ARTESIA FREEWAY',
 'HARBOR BOULEVARD',
 'ESCONDIDO FREEWAY',
 'MORENO VALLEY FREEWAY',
 'ORTEGA HIGHWAY',
 'LOS ALAMOS ROAD',
 'CAMPUS DRIVE',
 'GUINIDA LAN

In [35]:
hour,day,l = list(range(0,24)),list(range(0,7)),[]
for item in hour:
    for stuff in day:
        l.append(str(item)+'-'+str(stuff))
moments = pd.DataFrame(l)
moments = moments.rename(columns={0:'moment'})
moments

,moment
0,0-0
1,0-1
2,0-2
3,0-3
4,0-4
...,...
163,23-2
164,23-3
165,23-4
166,23-5


In [40]:
from progressbar import ProgressBar
pbar = ProgressBar()

def make_accident_free_rows(df, moments, road_name):
    tmp = df[df['OSRM_API_ROAD_name'] == road_name]
    tmp = tmp.merge(moments, how='outer')
    tmp['OSRM_API_ROAD_name'] = road_name
    return tmp
output=[]
for road in pbar(roads): # for each unique road name (in API)
    output.append(make_accident_free_rows(b, moments, road))
test = pd.concat([item for item in output])

100% |########################################################################|


In [42]:
test

case_id               3487305
OSRM_API_ROAD_name          0
dataset_ROAD_name     3487305
weather_1             3487305
collision_severity    3487305
latitude              3487305
longitude             3487305
hour                  3487305
weekday               3487305
moment                      0
dtype: int64

In [43]:
test.shape

(4205968, 10)

In [44]:
test.to_csv("moments_routes", index=False)

# Imputing the missing values thanks to a KNN imputer

In [ ]:
from sklearn.impute import KNNimputer
knnimputer = KNNimputer()


# checking if there is a pattern among missing lon/lat in the OSRM API vs initial dataset

In [ ]:
ds = pd.read_csv('../raw_data/csv/la_clean.csv')
ds.head()

In [ ]:
# let us fetch all rows in kaggle dataset that do not have a match in OSRM dataset:
missing = ds[~ds.case_id.isin(df.case_id)]
missing

In [ ]:
# plotting the lat/long of missing datapoints


missing.plot.scatter(x='longitude', y='latitude',figsize=(14,16),s=1)

In [ ]:
# no pattern, just L.A. datapoints